In [1]:
# %matplotlib inline
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt

import pickle
import os
import tensorflow as tf
import PIL
import IPython
import cv2
import time

from metrab_utils import *

def showarray(a, fmt='jpeg'):
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

model_fold = '/disk/apps/metrabs/models'

models = os.listdir(model_fold)
print(models)

['metrabs_rn18_y4', 'metrabs_eff2l_y4_360', 'metrabs_eff2s_y4', 'metrabs_mob3s_y4t', 'server.log', 'metrabs_mob3s_y4', 'metrabs_eff2l_y4']


In [2]:
mod = 'metrabs_mob3s_y4t' #10fps on rtx2070s
#mod = 'metrabs_rn18_y4' #9fps on rtx2070s

t = time.time()
model = tf.saved_model.load(os.path.join(model_fold, mod))
t = time.time() - t
print('load took %.2fs'%t)

2021-12-09 10:09:21.940663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 10:09:21.972474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 10:09:21.973114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 10:09:21.974404: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

load took 21.29s


In [2]:
cam = cv2.VideoCapture(0)

try:
    while(True):
        t1 = time.time()
        
        # Capture frame-by-frame
        ret, frame = cam.read()

        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        pred = model.detect_poses(frame)
        
        t2 = time.time()
        
        showarray(frame)

        print("%f FPS, %.2fms" % (1/(t2-t1), (t2-t1)*1000))
        # Display the frame until new frame is available
        IPython.display.clear_output(wait=True)
except KeyboardInterrupt:
    print("Stream stopped")
cam.release()

NameError: name 'model' is not defined

In [23]:
import json
data = json.dumps({"signature_name": "serving_default", "image": frame.tolist()})
#print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

In [25]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
#predictions = json.loads(json_response.text)['predictions']


In [26]:
json_response

<Response [400]>